## Question 1 
### a) Implémenter la structure de données du graphe de Brujin à l’aide d’une table de hachage qui devra également être implantée. Le constructeur prendra en entrée un itérable sur des chaînes de caractères de longueur k.
Puisque l’itérable est de longueur arbitraire, vous devez redimensionner dynamiquement la table de hachage pour maintenir le facteur de charge sous 0.75.

### Answer a)
参考graph.py文件

### b) Discuter des détails de votre implémentation en répondant aux questions suivantes
- Quelle fonction de hachage avez-vous utilisé? et dans quelle mesure est-elle spécifique au
problème?
- Quelle stratégie d’adressage avez-vous utilisé?

### Answer b)
- 简单介绍我们的hash_function(hash_code 和 compress_methods), 介绍hash_code具有特定长度的mers下具有编码唯一性，hash_code不存在冲突。
- 寻址技术: 采用probing的方式，并设计了多种probing方法:linear , quadratic, double-hash,初步的试验支持double-hash冲突最小

### c) Quel serait le principal gain d’utiliser une technique de hachage sensitif à la localité et de l’adressage linéaire pour ce problème?
### Answer c)
使用哈希敏感的寻址方式可以减少索引冲突发生的概率，进而提高运算速度，这在问题规模很大的情况下其对速度的提升尤其明显。

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("drive/My Drive/Colab Notebooks/IFT2015/")
#check if you are in the right directory
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
 contigs.fa
 contigs.fa.gz
 contigs_final.fa
 example.bed
 fast_occurences.bed
 GCF_000002985.6_WBcel235_rna.fna.gz
 graph_yq_1221.py
 graph_yq.ipynb
 graph_yq.py
 ift2015-travail-pratique-2.pdf
'Introduction aux formats de fichiers et algorithmes utilisés dans le TP2_copy.ipynb'
 main_yq_1220.ipynb
'main_yq (1).ipynb'
 Note_BinarySearchTree.ipynb
 occurences.bed
 occurences_final.bed
 __pycache__
 reads.fastq.gz
 TP2_JX_colab.ipynb


In [0]:
import sys
def show_live_info(info, desc_before = "", desc_after = ""):
    info_line = '\r' + desc_before +" " + str(info) + " " + desc_after + "\t"
    sys.stdout.flush()
    sys.stdout.write(info_line)

## Question 2

### a) Encoder tous les k-mers des fragments fournis dans le fichier FASTQ dans une instance du graphe De Brujin. Chaque fragment possède l − k + 1 k-mers.

### Answer a)
**Step1**: Import `DeBrujinGraph` implemented in question1 and other necessary modules

In [0]:
import gzip
from itertools import islice
from graph_yq import DeBrujinGraph
from graph_yq import print_progress, debug
import time

**Step2**: Use the method `read_fastq` provided to load sequences from file: 'reads.fastq.gz' and generate possible k-mers with k = 21.

In [0]:
def read_fastq(path):
    with gzip.open(path, 'rt') as f:
        for line in f:
            seqid, description = line[1:].rstrip().split(maxsplit = 1)
            sequence = f.readline().rstrip()
            _ = f.readline()
            quality = f.readline().rstrip()
            yield seqid, description, sequence, quality

In [0]:
k = 21
kmers = []

In [6]:
%%time
for _, _, seq, _ in read_fastq('reads.fastq.gz'):
    kmers.extend([seq[i:i+k] for i in range(len(seq) - k + 1)])
   
print(len(kmers), "k-mers")

8627600 k-mers
CPU times: user 2.47 s, sys: 564 ms, total: 3.03 s
Wall time: 3.04 s


**Step3**: Build an instance of `DeBrujinGraph` based on all k-mers.

In [7]:
%%time
length = len(kmers) 
#length = min(100000, len(kmers))
print(length)
graph = DeBrujinGraph(nodes = kmers[:length], k = 21)

8627600
building nodes...
building edges...
CPU times: user 30min 59s, sys: 13.1 s, total: 31min 12s
Wall time: 31min 12s


**Step4**: View the node and edge counts of the graph generated.

In [8]:
print("node counts:", graph.node_count())
print("edge counts:", graph.edge_count())

node counts: 5126513
edge counts: 5091467


### b) Bonus: Implanter une table de hachage plus rapide que celle de CPython (i.e. celle utilisée par set et dict) pour stocker des k-mers pour les opérations effectuées en 2a.

### Answer b)

python内置的dict使用的是动态扩展字典大小的方式，对于此问题，由于我们实现知道所要构建的图的节点规模，可以一次设定图的大小进而减少频繁扩展图大小的操作，以此来提高速度。同时尽可能的精简代码。

同时敏感的哈希函数能够减少索引冲突的可能性。

<hr>

## Question 3 

### a)  Parcourir le graphe pour obtenir des segments contigüs que vous devrez stocker dans un fichier FASTA nommé contigs.fa avec un identifiant unique pour chacun. 
Commencer par identifier les noeuds qui n’ont pas de prédécesseurs pour débuter l’exploration.

Alternativement, vous pouvez explorer le graphe dans les deux directions en partant d’un point aléatoire sur chaque composante connexe.

### Answer a)

**Step1**: Implement deep first search and breadth first search methods `DFS`, `DFS_complete`, `BFS`; these methods will be used to explore the connected components of the graph.

In [0]:
def DFS(g, u, discovered, dest_nodes):
    """deep first search of a graph.
    params
        g: a graph
        u: a node of graph g
        discovered: a dictionary marking where a edge is visited
        dest_nodes: a set of nodes who hae no successors in connected tree.
    return
        None, discovered and dest_nodes will be modified
    """
    for v in g.successors( u ):
        if v not in discovered:
            discovered[v] = u # u represents parent node of v
                              # v carries the edge information v[-1]
            dest_nodes.add(v)
            dest_nodes.discard(u)
            DFS( g, v, discovered, dest_nodes )
            
            
def BFS(g, s, discovered, dest_nodes):
    """deep first search of a graph.
    params
        g: a graph
        u: a node of graph g
        discovered: a dictionary marking where a edge is visited
        dest_nodes: a set of nodes who hae no successors in connected tree.
    return
        None, discovered and dest_nodes will be modified
    """
    level = [s]
    while len(level) > 0:
        next_level = []
        for u in level:
            for v in g.successors(u):
                if v not in discovered:
                    discovered[v] = u
                    dest_nodes.add(v)
                    dest_nodes.discard(u)
                    next_level.append(v)
        level = next_level  

**Step2**: Implement method `find_nodes` for finding nodes without predecessors or without successors.

In [0]:
def find_end_nodes(g: DeBrujinGraph, is_start: bool = True):
    """find all nodes without predecessors or without successors controled by
    a bool parameter
    params
        g: graph
        is_start: bool, if True, find all nodes without predecessors; if False
            find all nodes without successors
    returns
        results, list of nodes
    """
    results = []
    func = g.predecessors
    if is_start == False:
        func = g.successors
    for node in g.nodes():
        if len(func(node)) == 0: # no predecessors
            results.append(node)
            show_live_info(len(results), "searching... ", "nodes found.")
    print()
    return results           

In [11]:
%%time
start_nodes = find_end_nodes(graph, is_start = True)

searching...  37319 nodes found.	
CPU times: user 5min 51s, sys: 14.6 s, total: 6min 6s
Wall time: 6min 20s


**Step3**: We can find all possible paths by starting from a node without predecessors and ended to its accessible node who has no successors or one of its successors is its predecessor . Based on the paths found, a continuous sequence will be assembled.

In [0]:
def generate_sequence(g, u, v, discovered):
    """generate a contiguous sequence from node u to v based on the 
    discovered paths in graph
    params
        g: graph
        u: start(origin) node of a sequence, str
        v: end(destination) node of a sequence, str
        discovered: dictionary from a deep/breadth first search of a graph g
    returns
        segment: str, a string format of list of nodes. 
            example: "ACTGTCTATAT"
    """
    segment = []
    if u is None: # there is no start nodes
        return None   
    
    if v in discovered:
        walk = v
        while walk is not u:
            segment.append(walk[-1])
            walk = discovered[walk]
        # walk is now u
        
    segment.reverse()
    segment = u + "".join(segment)
    return segment


def generate_sequences(g, start_nodes):
    """generate all possible contiguous sequence from a directed graph with
    start nodes are given
    params
        g: graph
        start_nodes: list of start nodes, str
    returns
        segments: all possible contigous sequences, [str]
    """
    segments = []
    n_total_nodes = len(start_nodes)
    for i, start_node in enumerate(start_nodes):
        discovered = {start_node: '-'}
        dest_nodes = set()
        dest_nodes.add(start_node)
        # use DFS or BFS to construct a connected tree, and all leave nodes
        # circle path will be cut here.
        DFS(graph, start_node, discovered, dest_nodes)
        #BFS(graph, start_node, discovered, dest_nodes)
        for dest_node in dest_nodes:
            #print(start_node," -> ", dest_node)
            segment = generate_sequence(graph, start_node, dest_node, discovered)
            if segment is not None:
                segments.append(segment)
        
        show_live_info(len(segments), "generating sequence...", "generated.")
    print()
    return segments

**Step4**: Generate sequences assembled and write them to a file.

In [13]:
%%time
import sys
# sys.getrecursionlimit()
sys.setrecursionlimit(10000)
segments = generate_sequences(graph, start_nodes)

generating sequence... 51391 generated.	
CPU times: user 6min 45s, sys: 14.2 s, total: 6min 59s
Wall time: 7min 10s


In [0]:
import random
import string
def get_identifier(k = 10):
    return ''.join(random.choices(string.ascii_uppercase, k = 10))

def read_fasta(path):
    """read data from a fasta file. if the filename ends with 'gz', it will
    be considered a compressed file; otherwise, non compressed
    params
        path: file path, str
    returns
        None
    """
    open_style = gzip.open          # zp file as default 
    if path[-2:] != 'gz':
        open_style = open           # if not, use open
    with open_style(path, 'rt') as f:
        accession, description, seq = None, None, None
        for line in f:
            if line[0] == '>':
                # yield current record
                if accession is not None:
                    yield accession, description, seq
                    
                # start a new record
                accession, description = line[1:].rstrip().split(maxsplit=1)
                seq = ''
            else:
                seq += line.rstrip()
                
    return


def write_fasta(path, sequences):
    """write sequences data from a fasta file. if the filename ends with 'gz', 
    it will be considered a compressed file; otherwise, non compressed.
    params
        path: file path for writing, str
        sequences: list of sequence, [str]
    returns
        None
    """
    open_style = gzip.open
    if path[-2:] != 'gz': # not a compressed file
        open_style = open
    random.seed(123)
    desc_base = "Assembled by Qiang Ye and Ju Xiao 12/21/2018"
    identifiers = set()            # avoid repeatative identifiers
    max_bases_line = 80            # 80 bases maximal per line
    with open_style(path, "wt") as f:
        for sequence in sequences:
            identifier = get_identifier()
            while identifier in identifiers:  # assure each identifier is unique
                identifier = get_identifier()

            identifiers.add(identifier)
            desc = desc_base + " " + str(len(sequence)) + " bases"
            f.write(">" + identifier + " " + desc + "\n")
            echos = len(sequence) // max_bases_line
            if len(sequence) % max_bases_line > 0:
                echos += 1
            for echo in range(echos):
                start_index = echo * max_bases_line
                end_index = min(start_index + max_bases_line, len(sequence))
                f.write(sequence[start_index: end_index]+"\n")
    return

In [16]:
fasta_file = 'GCF_000002985.6_WBcel235_rna.fna.gz'
contigs_file = 'contigs_demo.fa'

write_fasta(contigs_file, segments)

print(next(read_fasta(fasta_file)))
print(next(read_fasta(contigs_file)))

('NM_001025782.2', 'Caenorhabditis elegans Phosphatidylinositol 3-kinase catalytic subunit type 3 (vps-34), partial mRNA', 'ATGCGAGTCTCCACGTCAGTCAACGGTGGTGTTGGGATTGTTTCGGCCTGTACCCGTTACTGTGTAGCTGATCCTGAACTTCTTCTAGAATCTCTGGCGGAAGTTAAACACAGTGCAATGACACGTCGAATCAGAGATGTTGAAGATGAGAGACATCGACAAGTAAAACCGAATAAACAAGCAAAGGATCGATTGGAGACTATTGTGAATCTTCCATCATCTCAAGTTCTTACAAGAGAACAAAGAGATTTAGTATGGAAATTCAGGCATTATTTAAGACAATTTCCAAAAGCTTTGAACAAATATCTACGATCAGTTAATTGGGTTCATCCACAAGAAGTAAAAACTGCATTAGCTCTGATGAATGATTGGGAGCTGATTGAAGCTGAAGATGCATTAGAATTACTTTCAAGTGCATTCACTCATCCGGCAGTTCGTGCATATTCTGTATCCCGTCTTCTAGAAGCTGCTTCTCCTGAGCAAGTCCTCCTATATCTTCCACAATTGGTTCAAGCTTTGAAATACGAACAAGGTCAACAACTGCCTGAAGAAGGAAATCCAGTGCCGGTGGTTTCTGAAGAAGAGGGAAAGATCCCATCAGTTGCAACTACGCCAACTGAAGAGCTAGAAGGACGAGATATGACAGTGGTTACGAAGAAAGAGGCTAGAAAGGCGGCTAGTGGAGATTTGGCCACCTTCCTGATTGACTATGCTCTTGCATCTCCAAAAGTATCCAACTATCTCTACTGGCATCTAAAAACTGAAATTGAATCAACAAAAGAAAGTAAAGAAGAACACTCGAAAATGTATCAAAATATCCAGGATAGATTGATGGAAGCACTTGTAAAGAGACCTGATACCCGTGCACAAGTTGAT

The shortest and largest lengths are shown:

In [17]:
lens = [len(segment) for segment in segments]
print(min(lens), max(lens))

24 5955


### b) Dans votre rapport, répondez aux questions suivantes:
- Quelle stratégie de parcours avez-vous adoptée?
- Comment les boucles sont-elles gérées durant le parcours?

### Answer b)
- 深度优先搜索(或者广度优先搜索)
- 根据深度有限搜索，如果结点之间构成一个环，那么深度优先搜索会切开这个环。假设存在一个结点u是构成环的其中一个结点，并且它是从某一个没有祖先的节点开始的深度优先搜索最先访问到的环中的点，如果存在环中另一个结点v，它的后继之一是u的话，那么将把该环从v到u的边切开。

## Question 4 

### a) Comparer les séquences assemblées avec les séquences de références contenus dans le fichier FASTA. 

Vous pouvez utiliser string.find pour effectuer chaque paire de comparaison. Reporter chaque
occurence dans un fichier occurences.bed contenant les quatre (4) colonnes suivantes séparées
par des tabulations:
 - reference: identifiant du segment de référence
 - start: début de l’occurence basé sur 1 inclusif (une occurence à l’indice zéro devrait être en
position 1)
 - end: fin de l’occurence basé sur 1 inclusif
 - contig: identifiant du contig
 
 
 ### Answer a)
 **Step1**: implement methods to wirte mathes to file and to read them out.

In [0]:
def read_bed(path):
    with open(path) as f:
        for line in f:
            ref, start, end, name = line.rstrip().split('\t')
            yield ref, int(start) - 1, int(end), name
        
def write_bed(path, occurences):
    with open(path, "w") as f:
        for occur in occurences:
            line = ""
            for item in occur:
                line += str(item) + "\t"
            f.write(line+"\n")
    return

**Step2**: Compare sequences in assembeld fasta and references fasta.

In [25]:
%%time
fasta_file = 'GCF_000002985.6_WBcel235_rna.fna.gz'
contigs_file = "contigs.fa"
occurences = []
failures = []
for identifier, _, assem_seq in read_fasta(contigs_file):
    found = False
    for accession, _, seq in read_fasta(fasta_file):
        index = seq.find(assem_seq)
        if index > -1:
            found = True # this assembled seq has been found in references
            result_line = accession, index+1, index+len(assem_seq), identifier
            occurences.append(result_line)
            show_live_info(str(len(occurences)) + " / " + str(len(failures)),
                       desc_after = "  (matches / failures)")

    if not found: # assembled seq didn't match any sequence in fasta 
        failures.append((identifier, assem_seq))
        show_live_info(str(len(occurences)) + "/" + str(len(failures)),
                       desc_after = "  (matches / failures)")

 3968 / 3   (matches / failures)	CPU times: user 17min 51s, sys: 12.7 s, total: 18min 4s
Wall time: 18min 10s


**Step3**: Wirte the result to file.

In [26]:
print()
occurences_file = "occurences_demo.bed"
write_bed(occurences_file, occurences)

Let's show the first 10 lines in the "occurences.bed"

In [27]:
count = 0
for result in read_bed(occurences_file):
    print(result)
    count += 1
    if count > 10:
        break

('NM_067605.4', 927, 1027, 'BCKCXANIWE')
('NM_065405.5', 368, 468, 'IIGALCPBIL')
('NM_059957.3', 18, 118, 'XCDUAXOGVT')
('NM_170904.2', 18, 118, 'XCDUAXOGVT')
('NM_071419.2', 963, 1063, 'IUFPNVIKUN')
('NM_001361852.1', 1276, 1376, 'RQRXMRJATE')
('NM_001361853.1', 1291, 1391, 'RQRXMRJATE')
('NM_001361854.1', 1309, 1409, 'RQRXMRJATE')
('NM_001029569.2', 1164, 1264, 'VXREIKBLOI')
('NM_001029570.2', 1164, 1264, 'VXREIKBLOI')
('NR_002425.1', 1170, 1270, 'VXREIKBLOI')


### b) Peut-il exister des séquences qui ne peuvent pas être assemblée correctement dans la référence (avec k = 21)? Le cas échéant, en fournir un exemple.

### Answer b)
先讲一下我们理解的没有正确组装的含义：即存在一个通过遍历图形成的从一个起始点到一个终点序列，该序列在reference fasta文件所包含的所有序列中不能完美匹配。
When performing comparison, we maintain a list recording those assembled sequences who are not successfully matched. We can check if this list has contents.

In [30]:
if len(failures) == 0:
    print("all assembled sequences are correctly assembled")
    print("each of them matches a or part of a sequence in FASTA")
else:
    print("sequences who didn't match any references are listed here:")
    for identifier, assem_seq in failures:
        print(identifier, assem_seq, len(assem_seq))

sequences who didn't match any references are listed here:
XDRLYWHUCP AGCAGTTTGGGTATCAGCAGTATTCACAGCATCCTCAGCAACATCCTCAGCAGCATCCTCAGCAGCAGCAACAAGTCTGGAATCCGAA 88
JSBNIUKMIY TTTCTATTAGGCTTAGGCTCAGCCTTAGGCTTAGGCTTAGGCTTAGCAGCAGGCCCAGGCTCAG 64
ZIRXSZSNSG TTTCTATTAGGCTTAGGCTCAGCCTTAGGCTTAGGCTTAGGCTTAGGCT 49


### c) Bonus: 
Proposer une structure de données appropriée pour effectuer une comparaison efficace des séquences assemblées avec celles de référence. La complexité doit être strictement inférieure à la méthode proposée en 4a.

### Answer c)

还在寻找，暂时还没找到更快的匹配方法